## 과제 2: SageMaker Clarify 사용
	
이 실습에서는 Amazon SageMaker Clarify를 사용하여 훈련 전 데이터와 훈련 후 모델에서 편향을 탐지하고 설명 가능성 보고서에 액세스합니다.


### 과제 2.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sagemaker_datawrangler
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

그런 다음, 데이터 집합을 가져오고 분할하고 업로드합니다.

In [ ]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

이제 XGBoost 모델을 훈련합니다. 이렇게 훈련한 모델을 SageMaker Clarify ModelConfig에 사용합니다. 훈련을 완료하려면 약 4-5분이 소요됩니다.

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

### 과제 2.2: SageMaker Clarify 작업 설정 및 편향 보고서 액세스

[SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html)를 사용하려면 먼저 훈련 작업에서 모델을 생성해야 합니다. 그런 다음, 필요한 구성을 설정하고 훈련된 모델에서 SageMaker Clarify를 실행합니다. 이 과제에서는 다음 작업을 완료합니다.

- 모델 생성
- SageMaker Clarify를 사용하도록 설정
- 편향 보고서 실행
- 보고서 액세스

### 과제 2.3: 모델 생성

SageMaker Clarify에 사용할 모델을 훈련 작업에서 생성합니다.

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### 과제 2.4: SageMaker Clarify를 사용하도록 설정

이제 SageMaker Clarify를 사용하도록 설정하여 구성을 시작합니다.

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

SageMaker Clarify를 사용하면 다양한 지표를 통해 모델 훈련 전에 데이터에 포함되어 있을 수 있는 편향 그리고 훈련 후에 모델에 포함되어 있는 편향을 각각 탐지할 수 있습니다. Clarify 분석 구성에 관한 자세한 내용은 [지표](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html)를 참조하세요.

SageMaker Clarify로 데이터 I/O 정보를 전달하도록 **DataConfig** 객체를 구성합니다. 여기서는 입력 데이터 집합 위치, 출력을 저장할 위치, 소득(**label**) 열, 헤더 이름 및 데이터 집합 유형을 지정합니다.

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

훈련된 모델 관련 정보를 전달하도록 **ModelConfig** 객체를 구성합니다. 모델 이름을 설정하고 프로덕션 모델로의 추가 트래픽 방지를 위한 임시 전용 엔드포인트를 설정합니다(**instance_type** 및 **instance_count**). 또한 엔드포인트 응답 페이로드 형식을 나타내는 **accept_type**과 엔드포인트에 대한 요청의 페이로드 형식을 나타내는 **content_type**도 설정합니다.

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

샘플 확률을 편향 분석용 이진 레이블로 변환하도록 **ModelPredictedLabelConfig**의 **probability_threshold**를 구성합니다. 임계값보다 큰 예측은 레이블 값 **1**로 해석됩니다. 임계값 이하인 경우 레이블 값 **0**으로 해석됩니다.

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

민감한 열(**facets**), 잠재적인 민감한 특성(**facet_values_or_threshold**) 및 적절한 결과(**label_values_or_threshold**)를 나타내도록 **BiasConfig**를 구성합니다.

**facet_values_or_threshold** 및 **label_values_or_threshold**에는 범주형 데이터와 연속형 데이터를 모두 지정할 수 있습니다. **sex** 및 **age** 특성은 범주형 데이터입니다.

이 모델은 예측 대상의 소득이 미화 5만 달러 이상인지를 결정해야 합니다. 결과는 양수로 반환되어야 합니다. **BiasConfig**를 사용해 민감한 그룹(**Sex**) 관련 패싯이 포함된 열 관련 정보를 제공합니다. 즉, 모델에서 사용되었을 수 있는 민감한 특성(**facet_values_or_threshold**)과 모델에 사용되고 있는 적절한 결과(**label_values_or_threshold**)를 지정합니다. 결과를 **age**별로 그룹화하여 대상의 연령에 따라 결과가 달라지는지를 확인합니다.

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### 과제 2.5: 편향 보고서 실행

훈련 전 및 훈련 후 분석용 구성을 사용하여 편향 보고서를 생성합니다. 이 단계를 수행하려면 약 15-20분이 소요됩니다. 편향 보고서가 생성되는 동안 SageMaker Clarify가 [금융 업계의 기계 학습용 공정성 측정값](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf)을 계산하는 방식을 살펴볼 수 있습니다.

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### 과제 2.6: 편향 보고서 액세스

SageMaker Studio의 Experiments 탭에서 결과를 확인할 수 있습니다.

다음 단계에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_8_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **lab_8_ko_kr.ipynb** 탭을 선택한 다음, 도구 모음에서 **File**과 **New View for Notebook**을 선택합니다. 그러면 편향 보고서를 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 편향 보고서 탐색을 완료한 후 **lab_8_ko_kr.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. SageMaker Studio에서 **SageMaker Home** 아이콘을 선택합니다.

2. **Experiments**를 선택합니다.

SageMaker Studio에서 **Experiments** 탭이 열립니다.

3. **Experiments** 탭의 왼쪽에서 **Unassigned runs**를 선택합니다.

4. **Unassigned runs** 목록에서 제목에 **clarify-bias-** 가 포함된 훈련 작업 이름을 선택합니다. 

5. **Experiments** 탭의 왼쪽에서 **Bias Reports**를 선택합니다.

지표의 편향 보고서는 SageMaker Clarify가 편향 작업 실행을 완료하면 사용할 수 있습니다.

6. SageMaker Clarify가 편향 작업 실행을 완료할 때까지 기다립니다.

각 편향 지표에는 예제가 포함된 자세한 설명이 있습니다. 각 값을 선택하면 이러한 예제를 살펴볼 수 있습니다. 

7. 편향 지표 옆의 화살표를 선택하고 필드를 확장하여 **Class Imbalance** 및 **Disparate (Adverse) Impact (DI)** 세부 설명을 검토합니다. 

편향 지표 탐색을 완료한 후 다음 과제를 계속 진행합니다.

### 과제 2.7: 설명 가능성 보고서 액세스

SageMaker Clarify에서는 편향 보고서를 사용할 수 있을 뿐 아니라 개별 예측의 논리적 설명도 분석할 수 있습니다. 여기서는 보고서를 생성하여 SageMaker Clarify에서 생성한 예측의 설명 가능성 결과를 검토하고 보고서의 주요 지표를 분석합니다. 이 과제에서는 다음 작업을 완료합니다.

1. Shapley 값(SHAP) 구성 정의
2. 설명 가능성 보고서 실행
3. 설명 가능성 보고서 액세스

### 과제 2.8: SHAP 구성 정의

설명 가능성 보고서를 실행하여 하나의 모델이 특정 예측을 수행한 이유에 관한 설명을 확인할 수 있습니다. SageMaker Clarify는 SHAP를 사용하여 모델의 결정 추론에 관한 보고서를 생성합니다. SHAP 기준에 관한 자세한 내용은 [설명 가능성을 위한 SHAP 기준](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)을 참조하세요.

구성되는 SHAP 지표는 다음과 같습니다.
- **baseline**: 커널 SHAP 알고리즘에서 기준 데이터 집합으로 사용할 행 목록, 즉 Amazon Simple Storage Service(Amazon S3) 객체 URI입니다.
- **num_samples**: 커널 SHAP 알고리즘에서 사용할 샘플의 수입니다. 이 값에 따라 SHAP 값을 계산하기 위해 생성되는 합성 데이터 집합의 크기가 결정됩니다.
- **agg_method: mean_abs**: 모든 인스턴스의 절대 SHAP 값 평균입니다.
- **save_local_shap_values**: 로컬 SHAP 값을 출력 위치에 저장할지를 지정하는 부울 값입니다.

사용된 지표에 관한 자세한 내용은 [SHAP 지표](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html)를 참조하세요.

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### 과제 2.9: 설명 가능성 보고서 실행

이전 과제에서 지정한 구성을 사용하여 설명 가능성 보고서를 생성합니다. 이 단계를 수행하려면 약 10-15분이 소요됩니다. 설명 가능성 보고서가 생성되는 동안 다음 과제를 계속 수행하면 SageMaker Studio에서 작업 상태를 따라 진행할 수 있습니다.

SageMaker Clarify 설명 가능성 프로세스에 관한 자세한 내용은 [Amazon AI 공정성 및 설명 가능성 백서](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)를 참조하세요.

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### 과제 2.10: 설명 가능성 보고서 액세스

편향 보고서와 마찬가지로 Studio의 Experiments에 속한 설명 가능성 보고서를 검토할 수 있습니다.

1. **Experiments** 탭을 선택합니다.

2. **Experiments** 탭의 왼쪽에서 **Unassigned runs**를 선택합니다.

3. **Unassigned runs** 목록에서 제목에 **clarify-bias-** 가 포함된 훈련 작업 이름을 선택합니다. 

4. **Experiments** 탭의 왼쪽에서 **Explainability**를 선택합니다.

5. 설명 가능성 작업이 완료된 후 **Feature Importance** 차트를 확인하려면 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**View sample notebook**</span>를 선택합니다.

SageMaker Studio 탭에서 **Fairness and Explainability with SageMaker Clarify** 노트북이 열립니다.

**참고**: 이 샘플 노트북을 실행할 필요는 없습니다. 이 샘플 노트북은 이미 실행되었습니다.

중요도가 가장 높은 특성과 가장 낮은 특성은 각각 무엇인가요? 예상치 못한 결과가 있나요?

### 마무리

축하합니다! SageMaker Clarify를 사용하여 신뢰도와 규정 준수 수준이 더욱 높은 모델을 개발하는 데 사용할 수 있는 편향 및 설명 가능성 보고서를 생성했습니다. 다음 실습에서는 모델을 배포하고 추론을 실행합니다. 다음 실습에서도 이 모델을 계속 사용합니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 지침으로 돌아갑니다.